In [1]:
import cv2
import threading
import time
from ultralytics import YOLO

In [2]:
# Initialize global variables
current_frame = None
processed_frame = None

In [3]:
# Load the YOLO model
model = YOLO('trained_models/yolov8s.pt')

In [4]:
def process_frame():
    global current_frame, processed_frame
    while True:
        if current_frame is not None:
            try:
                # Process the current frame with YOLO
                results = model(current_frame, verbose=False)
                # Extract the annotated frame
                annotated_frame = results[0].plot()
                processed_frame = annotated_frame
    
            except Exception as e:
                print(f"Error processing frame: {e}")
            finally:
                time.sleep(1)  # Simulate a delay

In [5]:
def display_video():
    global current_frame, processed_frame
    cap = cv2.VideoCapture('videos/video_1.mp4')
    fps = 30

    try:
        # Create a thread for processing frames
        threading.Thread(target=process_frame, daemon=True).start()

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            current_frame = frame
            
            if processed_frame is not None:
                # Overlay the processed (annotated) frame onto the original frame
                frame = cv2.addWeighted(frame, 0.5, processed_frame, 0.5, 0)

            cv2.imshow('Video', frame)
            
            if cv2.waitKey(fps) & 0xFF == ord('q'):
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()

In [6]:
display_video()